
# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con la demanda eléctrica.** Las provincias con un PIB más alto o en crecimiento experimentan una mayor demanda de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Extracción

En el laboratorio de hoy tendrás que extraer la información necesaria para obtener tu objetivo de las siguientes fuentes de datos (deberás usar API's y herramientas de *web scrapping*):

- **Datos de la API de Red Eléctrica Española (REE):** Deberás extraer datos mensuales a nivel provincial de los siguientes aspectos:

  - **Demanda Eléctrica:** Nos proporciona los datos de demanda eléctrica a nivel provincial, agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/demanda/evolucion", añadiendo los parámetros que sean necesarios. 

  - **Generación Eléctrica:** Nos proporciona los datos de generación eléctrica a nivel provincial, diferenciando entre fuentes de energía (eólica, solar, hidroeléctrica, etc.), agregados de manera mensual. Tendrás que usar el endpoint de "https://apidatos.ree.es/es/datos/generacion/estructura-renovables", añadiendo los parámetros que sean necesarios.

  La documentación de la API la encontrarás en [este link](https://www.ree.es/es/apidatos). Recuerda leer en detenimiento la documentación. 

- **Datos del Instituto Nacional de Estadística (INE):** Además de los datos de la REE, debes extraer y utilizar datos socioeconómicos de las siguientes páginas del INE:

- **Datos Demográficos:** Extraer los datos de población por provincias, diferenciando por grupos de edad, sexo, y extrajeros. Estos datos serán utilizados para analizar cómo la población afecta a la demanda eléctrica en cada provincia.

  - **Página web:** [INE - Población por provincias](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990)

  - "Principales series 1998-2022" --> "Por provincia" --> " Población por provincias, edad (3 grupos de edad), españoles/Extranjeros, Sexo y año"

- **Datos Económicos:**

  - **Página web:** [INE - PIB por provincias](https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426&capsel=10429). 

  - **Pasos para la extracción**:" Resultados provinciales. Serie contable 2016-2021" --> "P.I.B. a precios de mercado y valor añadido bruto a precios básicos por ramas de actividad: Precios corrientes por provincias y periodo."



NOTA1: Tienes que sacar muchos datos, pero recuerda que hemos aprendido herramientas de asincronia que te pueden ayudar en este paso de la ETL. 

NOTA2: Todos estos datos los debes sacar para los años 2019-2020-2021

In [3]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random
import time

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 
from selenium.webdriver.common.by import By

SCRAPPING INE

 **Datos Demográficos:** 

In [11]:
url = "https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736177012&menu=resultados&idp=1254734710990"

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": "C:\\Users\\alexc\\Desktop\\GIT\\5.1.ETL-Extraccion\\datos_demograficos",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)
sleep(2)
driver.find_element("css selector", "#aceptarCookie")
sleep(2)
driver.find_element("xpath", '/html/body/div[1]/main/section[2]/div[1]/div[1]/div[1]/ul/li/ul/li[2]/a').click()
sleep(2)
driver.find_element("xpath", "/html/body/div/main/div[2]/ul/li[3]/ul/li[1]/a").click()
sleep(2)
for n in range(1, 4):
    driver.find_element("xpath", f"//*[@id='tg{n}']/div/fieldset/div[2]/button[1]/i").click()
    sleep(1)
sleep(2)
driver.find_element("xpath", "/html/body/div/main/form/ul/li[1]/ul/li[5]/div/fieldset/div[3]/button[2]/i").click()
anios = driver.find_elements("css selector", "#periodo")
cant_anios = anios[0].find_elements(By.TAG_NAME, "option")
sleep(1)
for n in range(2, 5):
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="periodo"]/option[{n}]'))).click()
    sleep(1)

# Seleccionar el iframe para descargar csv
sleep(2)
try:
    driver.find_element("xpath", "/html/body/div[1]/main/form/div[5]/input[3]").click()
except:
    print("no encuentro el botón de consulta")
sleep(2)
try:
    driver.find_element("css selector", "#botonConsulSele").click()
except:
    print("no encuentro el botón de consulta")
sleep(2)
driver.find_element("xpath", "/html/body/div[1]/main/ul/li/div/div/form[2]/button/i").click()
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="thickBoxINEfrm"]')))
driver.switch_to.frame(iframe)
sleep(2)
driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
sleep(2)
driver.switch_to.default_content()
sleep(2)
driver.close()



no encuentro el botón de consulta
no encuentro el botón de consulta


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/main/ul/li/div/div/form[2]/button/i"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF650673AF5+28005]
	(No symbol) [0x00007FF6505D83F0]
	(No symbol) [0x00007FF65047580A]
	(No symbol) [0x00007FF6504C5A3E]
	(No symbol) [0x00007FF6504C5D2C]
	(No symbol) [0x00007FF65050EA97]
	(No symbol) [0x00007FF6504EBA7F]
	(No symbol) [0x00007FF65050B8B3]
	(No symbol) [0x00007FF6504EB7E3]
	(No symbol) [0x00007FF6504B75C8]
	(No symbol) [0x00007FF6504B8731]
	GetHandleVerifier [0x00007FF65096646D+3118813]
	GetHandleVerifier [0x00007FF6509B6CC0+3448624]
	GetHandleVerifier [0x00007FF6509ACF3D+3408301]
	GetHandleVerifier [0x00007FF65073A44B+841403]
	(No symbol) [0x00007FF6505E344F]
	(No symbol) [0x00007FF6505DF4

**Datos Económicos:**

In [25]:
url = "https://www.ine.es/dynt3/inebase/es/index.htm?padre=10426"

chrome_options = webdriver.ChromeOptions()

prefs = {
    "download.default_directory": "C:\\Users\\alexc\\Desktop\\GIT\\5.1.ETL-Extraccion\\datos_economicos",  # AQUÍ CADA UNO TENDREMOS QUE PONER LA RUTA QUE QUERAMOS PARA QUE SE GUARDEN LOS ARCHIVOS DESCARGADOS
    "download.prompt_for_download": False,   # desactiva el diálogo que Chrome normalmente muestra para pedir confirmación del usuario antes de descargar un archivo
    "directory_upgrade": True,    # hace que Chrome actualice el directorio de descarga predeterminado a la nueva ubicación especificada por download.default_directory si esta ha cambiado.
}

chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)

driver.get(url)
driver.maximize_window()
driver.implicitly_wait(5)
sleep(2)
try:
    driver.find_element("css selector", "#aceptarCookie")
except:
    print("no encuentro el botón de cookies")
sleep(2)
try:
    driver.find_element("css selector", '#c_10428').click()
except:
    print("no encuentro el botón de categoria")
sleep(2)
try:
    driver.find_element("xpath", '/html/body/div[1]/main/div[2]/ul/li[2]/ul/li[1]/a').click()
except:
    print("no encuentro el botón desubcar2")
sleep(2)
for n in range(1, 16):
    driver.find_element("xpath", f"/html/body/div[1]/main/form/ul/li[1]/ul/li[2]/div/fieldset/select/option[{n}]").click()
    sleep(1)
driver.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[2]/div/fieldset/select/option[1]").click()
sleep(2)
for i in range(1, 5):
    try:
        driver.find_element("xpath", f"/html/body/div[1]/main/form/ul/li[1]/ul/li[3]/div/fieldset/select/option[{i}]").click()
        sleep(1)
    except:
        print(f"no encuentro el botón {i}")
driver.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[3]/div/fieldset/select/option[1]").click()  

for j in range(2, 5):
    try:
        driver.find_element("xpath", f"/html/body/div[1]/main/form/ul/li[1]/ul/li[4]/div/fieldset/select/option[{j}]").click()
        sleep(1)
    except:
        print(f"no encuentro el botón {j}")

driver.find_element("xpath","/html/body/div[1]/main/form/ul/li[1]/ul/li[4]/div/fieldset/select/option[2]").click()  
sleep(2)
driver.find_element("xpath", "/html/body/div[1]/main/form/div[5]/input[3]").click()
sleep(2)
driver.find_element("css selector", "#botonConsulSele").click()
sleep(2)
driver.find_element("xpath", "/html/body/div[1]/main/ul/li/div/div/form[2]/button/i").click()
iframe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="thickBoxINEfrm"]')))
driver.switch_to.frame(iframe)
sleep(2)
driver.find_element("xpath", "/html/body/form/ul/li[4]/label").click()
sleep(2)
driver.switch_to.default_content()
sleep(2)
driver.close()

no encuentro el botón 4


ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" class="botonSel" id="botonConsulSele" name="consulsele" value="Consultar selección" onmouseover=""> is not clickable at point (675, 628). Other element would receive the click: <p>...</p>
  (Session info: chrome=130.0.6723.70)
Stacktrace:
	GetHandleVerifier [0x00007FF650673AF5+28005]
	(No symbol) [0x00007FF6505D83F0]
	(No symbol) [0x00007FF65047580A]
	(No symbol) [0x00007FF6504CD6CE]
	(No symbol) [0x00007FF6504CB16C]
	(No symbol) [0x00007FF6504C8628]
	(No symbol) [0x00007FF6504C785D]
	(No symbol) [0x00007FF6504B990E]
	(No symbol) [0x00007FF6504EBA3A]
	(No symbol) [0x00007FF6504B9246]
	(No symbol) [0x00007FF6504EBC50]
	(No symbol) [0x00007FF65050B8B3]
	(No symbol) [0x00007FF6504EB7E3]
	(No symbol) [0x00007FF6504B75C8]
	(No symbol) [0x00007FF6504B8731]
	GetHandleVerifier [0x00007FF65096646D+3118813]
	GetHandleVerifier [0x00007FF6509B6CC0+3448624]
	GetHandleVerifier [0x00007FF6509ACF3D+34083

In [19]:
list(range(2, 4))

[2, 3]

EXTRACCION API

In [14]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [15]:
ListIdcomunidades = list(cod_comunidades.values())

In [16]:
# Definir URLs de los endpoints
base_url_demanda = "https://apidatos.ree.es/es/datos/demanda/evolucion"
base_url_generacion = "https://apidatos.ree.es/es/datos/generacion/estructura-renovables"
headers = {'Accept': 'application/json'}

# Función para extraer datos de un solo año
def obtener_datos(tipo, anio, comunidad_id):
    base_url = base_url_demanda if tipo == 'demanda' else base_url_generacion
    fecha_inicio = f"{anio}-01-01"
    fecha_fin = f"{anio}-12-31"
    params = {'start_date': fecha_inicio,
            'end_date': fecha_fin,
            'time_trunc': 'month',
            'geo_limit': 'ccaa',
            'geo_ids': comunidad_id}
    response = requests.get(base_url, headers=headers, params=params)
    
    # Revisar si la solicitud fue exitosa
    if response.status_code == 200:
        data = response.json()
        valores = data['included'][0]['attributes']['values']
        df = pd.DataFrame(valores)
        df['Tipo'] = tipo
        df['Id_Comunidad']=comunidad_id
        return df
    else:
        print(f"Error {response.status_code} al solicitar {tipo} para {comunidad_id} en el año {anio}")
        return pd.DataFrame()

In [17]:
lista_años = list(range(2019, 2022))

listadf_demanda=[]
listadf_generacion=[]

for id in ListIdcomunidades:
    for año in lista_años:
        datos_demanda = obtener_datos('demanda', año, id)
        datos_generacion = obtener_datos('generacion',año, id)
        listadf_demanda.append(datos_demanda)
        listadf_generacion.append(datos_generacion)


Error 502 al solicitar generacion para 8744 en el año 2019
Error 502 al solicitar generacion para 8744 en el año 2020
Error 502 al solicitar generacion para 8744 en el año 2021


In [18]:
df_demanda=pd.concat(listadf_demanda,ignore_index=True)
df_generacion=pd.concat(listadf_generacion,ignore_index=True)

In [19]:
df_demanda.dtypes

value           float64
percentage        int64
datetime         object
Tipo             object
Id_Comunidad      int64
dtype: object

In [23]:
df_generacion.dtypes

value           float64
percentage      float64
datetime         object
Tipo             object
Id_Comunidad      int64
dtype: object

In [27]:
df_demanda.to_csv("./data/datosdemanda.csv")
df_generacion.to_csv("./data/datosgeneración.csv")